Code for Processing of IMU data for entire duration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack
from scipy.signal import butter, lfilter, freqz, filtfilt
import pandas as pd
import xlsxwriter 
import os
import glob
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


Ref=pd.read_excel(r'numpyref.xlsx')
if not os.path.isdir('PSD_IMU_Data'):
    os.mkdir('PSD_IMU_Data')
dir_list=glob.glob('Resampled_IMU_Head/*.xlsx')
dir_list.sort()
print(dir_list)
workbook = xlsxwriter.Workbook('PSD_IMU_Data/IMU_PSD_data.xlsx') 
row=0
bold = workbook.add_format({'bold': True})
PSD_Mean=np.array
for j in range(0,len(dir_list)):
    #worksheet = workbook.add_worksheet(Ref.OUTPUT[j]) 
    filename = dir_list[j]
    Metal=pd.read_excel(filename,'Head_Metal',header=0,names=None,index_col=0)
    Favourite=pd.read_excel(filename,'Head_Favourite',header=0,names=None,index_col=0)
    Dir=Ref.TITLE_SHEET[j]
    #os.mkdir('PSD_IMU_Data/'+Dir)
    param_full=Metal.columns.values
    writer = pd.ExcelWriter('PSD_IMU_Data'+'/'+Dir+'/'+Ref.OUTPUT[j])
    for i in range(1,16):
        if workbook.get_worksheet_by_name(param_full[i]):
            worksheet=workbook.get_worksheet_by_name(param_full[i])
        else:
            worksheet = workbook.add_worksheet(param_full[i])
            row=0
            worksheet.write(row,0,'Participant',bold)
            worksheet.write(row,1,'MPF_Metal',bold)
            worksheet.write(row,2,'MPF_Favourite',bold)
            worksheet.write(row,3,'(MPF_Favourite-MPF_Metal)',bold)
            worksheet.write(row,4,'Area_Metal',bold)
            worksheet.write(row,5,'Area_Favourite',bold)
            worksheet.write(row,6,'Max_Power_Metal',bold)
            worksheet.write(row,7,'Dom_Freq_Metal',bold)
            worksheet.write(row,8,'Max_Power_Favourite',bold)
            worksheet.write(row,9,'Dom_Freq_Favourite',bold)
            row+=1
        param=param_full[i]
        low_cutoff=0.5
        high_cutoff=10
        freq=100
        order=4
        PSD=plt.figure()
        Metal_PSD=plt.psd(butter_bandpass_filter(Metal[param], low_cutoff,high_cutoff, freq, order), Fs=100)
        Favourite_PSD=plt.psd(butter_bandpass_filter(Favourite[param],low_cutoff,high_cutoff, freq, order), Fs=100)
        plt.xlim((0,10))
        plt.ylim(min(np.amin(10*np.log10(Metal_PSD[0][Metal_PSD[1]<10])),np.amin(10*np.log10(Favourite_PSD[0][Favourite_PSD[1]<10]))),max(np.amax(10*np.log10(Metal_PSD[0][Metal_PSD[1]<10])),np.amax(10*np.log10(Favourite_PSD[0][Favourite_PSD[1]<10]))))
        PSD.legend(['Metal','Favourite'])
        PSD.suptitle(param)
        PSD.savefig('PSD_IMU_Data'+'/'+Dir+'/'+Ref.TITLE_SHEET[j]+'_'+param+'.png')
        MPF_Metal=np.linalg.norm(np.multiply(Metal_PSD[0][Metal_PSD[1]<10],Metal_PSD[1][Metal_PSD[1]<10]),1)/np.linalg.norm(Metal_PSD[0][Metal_PSD[1]<10],1)
        MPF_Favourite=np.linalg.norm(np.multiply(Favourite_PSD[0][Favourite_PSD[1]<10],Favourite_PSD[1][Favourite_PSD[1]<10]),1)/np.linalg.norm(Favourite_PSD[0][Favourite_PSD[1]<10],1)
        worksheet.write(row,0,Ref.OUTPUT[j])
        worksheet.write(row,1,MPF_Metal)
        worksheet.write(row,2,MPF_Favourite)
        worksheet.write(row,3,(MPF_Favourite-MPF_Metal))
        worksheet.write(row,4,np.linalg.norm(np.multiply(Metal_PSD[0],Metal_PSD[1]),1))
        worksheet.write(row,5,np.linalg.norm(np.multiply(Favourite_PSD[0],Favourite_PSD[1]),1))
        worksheet.write(row,6,Metal_PSD[0][np.argmax(Metal_PSD[0])])
        worksheet.write(row,7,Metal_PSD[1][np.argmax(Metal_PSD[0])])
        worksheet.write(row,8,Favourite_PSD[0][np.argmax(Favourite_PSD[0])])
        worksheet.write(row,9,Favourite_PSD[1][np.argmax(Favourite_PSD[0])])       
        Stresed=pd.DataFrame({'freq':Metal_PSD[1][Metal_PSD[1]<10],'Power':Metal_PSD[0][Metal_PSD[1]<10]})
        Relaxed=pd.DataFrame({'freq':Favourite_PSD[1][Metal_PSD[1]<10],'Power':Favourite_PSD[0][Metal_PSD[1]<10]})
        Stresed.to_excel(writer,'Stressed_Drive_'+param)
        Relaxed.to_excel(writer,'Relaxed_Drive_'+param)
        if not (j or i-1):
            Metal_PSD_Average=np.zeros((len(Metal_PSD[1][Metal_PSD[1]<10]),15))
            Favourite_PSD_Average=np.zeros((len(Favourite_PSD[1][Favourite_PSD[1]<10]),15))
        Metal_PSD_Average[:,i-1]+=10*np.log10(Metal_PSD[0][Metal_PSD[1]<10])
        Favourite_PSD_Average[:,i-1]+=10*np.log10(Favourite_PSD[0][Favourite_PSD[1]<10])
        plt.clf()
    writer.save()
    row=row+1
    plt.clf()
Metal_PSD_Average=Metal_PSD_Average/len(dir_list)
Favourite_PSD_Average=Favourite_PSD_Average/len(dir_list)
workbook.close() 
Metal_Average=pd.DataFrame(Metal_PSD_Average)
Metal_Average.columns=param_full[1:16]
Metal_Average.to_csv(r'PSD_IMU_Data/AverageMetalPSD.csv')
Favourite_Average=pd.DataFrame(Favourite_PSD_Average)
Favourite_Average.columns=param_full[1:16]
Favourite_Average.to_csv(r'PSD_IMU_Data/AverageFavouritePSD.csv')
for j in Metal_Average.columns:
    fig=plt.figure()
    plt.plot(Metal_PSD[1][Metal_PSD[1]<10],(Metal_Average[j]),'r')
    plt.plot(Metal_PSD[1][Metal_PSD[1]<10],(Favourite_Average[j]),'b')
    plt.legend(('Stressed','Relaxed'))
    plt.title(j)
    plt.xlabel('Frequency')
    plt.ylabel(j+'(in dB)')
    plt.savefig('PSD_IMU_Data/MeanPSD/mean_of_log/'+j+'.png')

Code for Processing of IMU data for first minute

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack
from scipy.signal import butter, lfilter, freqz, filtfilt
import pandas as pd
import xlsxwriter 
import os
import glob
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


Ref=pd.read_excel(r'numpyref.xlsx')
if not os.path.isdir('PSD_IMU_Data/onemin'):
    os.mkdir('PSD_IMU_Data/onemin')
dir_list=glob.glob('Resampled_IMU_Head/*.xlsx')
dir_list.sort()
print(dir_list)
workbook = xlsxwriter.Workbook('IMU_PSD_data_onemin.xlsx') 
row=0
bold = workbook.add_format({'bold': True})
PSD_Mean=np.array
for j in range(0,len(dir_list)):
    #worksheet = workbook.add_worksheet(Ref.OUTPUT[j]) 
    filename = dir_list[j]
    Metal=pd.read_excel(filename,'Head_Metal',header=0,names=None,index_col=0)
    Favourite=pd.read_excel(filename,'Head_Favourite',header=0,names=None,index_col=0)
    Dir=Ref.TITLE_SHEET[j]
    if not os.path.isdir('PSD_IMU_Data/onemin/'+Dir):
        os.mkdir('PSD_IMU_Data/onemin/'+Dir)
    param_full=Metal.columns.values
    writer = pd.ExcelWriter('PSD_IMU_Data/onemin'+'/'+Dir+'/'+Ref.OUTPUT[j])
    for i in range(1,16):
        if workbook.get_worksheet_by_name(param_full[i]):
            worksheet=workbook.get_worksheet_by_name(param_full[i])
        else:
            worksheet = workbook.add_worksheet(param_full[i])
            row=0
            worksheet.write(row,0,'Participant',bold)
            worksheet.write(row,1,'MPF_Metal',bold)
            worksheet.write(row,2,'MPF_Favourite',bold)
            worksheet.write(row,3,'(MPF_Favourite-MPF_Metal)',bold)
            worksheet.write(row,4,'Area_Metal',bold)
            worksheet.write(row,5,'Area_Favourite',bold)
            worksheet.write(row,6,'Max_Power_Metal',bold)
            worksheet.write(row,7,'Dom_Freq_Metal',bold)
            worksheet.write(row,8,'Max_Power_Favourite',bold)
            worksheet.write(row,9,'Dom_Freq_Favourite',bold)
            row+=1
        param=param_full[i]
        low_cutoff=0.5
        high_cutoff=10
        freq=100
        order=4
        PSD=plt.figure()
        Metal_PSD=plt.psd(butter_bandpass_filter(Metal[param], low_cutoff,high_cutoff, freq, order)[:6000], Fs=100)
        Favourite_PSD=plt.psd(butter_bandpass_filter(Favourite[param],low_cutoff,high_cutoff, freq, order)[:6000], Fs=100)
        plt.xlim((0,10))
        plt.ylim(min(np.amin(10*np.log10(Metal_PSD[0][Metal_PSD[1]<10])),np.amin(10*np.log10(Favourite_PSD[0][Favourite_PSD[1]<10]))),max(np.amax(10*np.log10(Metal_PSD[0][Metal_PSD[1]<10])),np.amax(10*np.log10(Favourite_PSD[0][Favourite_PSD[1]<10]))))
        PSD.legend(['Metal','Favourite'])
        PSD.suptitle(param)
        PSD.savefig('PSD_IMU_Data/onemin'+'/'+Dir+'/'+Ref.TITLE_SHEET[j]+'_'+param+'.png')
        plt.clf()
        MPF_Metal=np.linalg.norm(np.multiply(Metal_PSD[0][Metal_PSD[1]<10],Metal_PSD[1][Metal_PSD[1]<10]),1)/np.linalg.norm(Metal_PSD[0][Metal_PSD[1]<10],1)
        MPF_Favourite=np.linalg.norm(np.multiply(Favourite_PSD[0][Favourite_PSD[1]<10],Favourite_PSD[1][Favourite_PSD[1]<10]),1)/np.linalg.norm(Favourite_PSD[0][Favourite_PSD[1]<10],1)
        worksheet.write(row,0,Ref.OUTPUT[j])
        worksheet.write(row,1,MPF_Metal)
        worksheet.write(row,2,MPF_Favourite)
        worksheet.write(row,3,(MPF_Favourite-MPF_Metal))
        worksheet.write(row,4,np.linalg.norm(np.multiply(Metal_PSD[0][Metal_PSD[1]<10],Metal_PSD[1][Metal_PSD[1]<10]),1))
        worksheet.write(row,5,np.linalg.norm(np.multiply(np.multiply(Favourite_PSD[0][Favourite_PSD[1]<10],Favourite_PSD[1][Favourite_PSD[1]<10]),1),1))
        worksheet.write(row,6,Metal_PSD[0][np.argmax(Metal_PSD[0][Metal_PSD[1]<10])])
        worksheet.write(row,7,Metal_PSD[1][np.argmax(Metal_PSD[0][Metal_PSD[1]<10])])
        worksheet.write(row,8,Favourite_PSD[0][np.argmax(Favourite_PSD[0][Favourite_PSD[1]<10])])
        worksheet.write(row,9,Favourite_PSD[1][np.argmax(Favourite_PSD[0][Favourite_PSD[1]<10])])       
        Stresed=pd.DataFrame({'freq':Metal_PSD[1][Metal_PSD[1]<10],'Power':Metal_PSD[0][Metal_PSD[1]<10]})
        Relaxed=pd.DataFrame({'freq':Favourite_PSD[1][Metal_PSD[1]<10],'Power':Favourite_PSD[0][Metal_PSD[1]<10]})
        Stresed.to_excel(writer,'Stressed_Drive_'+param)
        Relaxed.to_excel(writer,'Relaxed_Drive_'+param)
        if not (j or i-1):
            Metal_PSD_Average=np.zeros((len(Metal_PSD[1][Metal_PSD[1]<10]),15))
            Favourite_PSD_Average=np.zeros((len(Favourite_PSD[1][Favourite_PSD[1]<10]),15))
        Metal_PSD_Average[:,i-1]+=10*np.log10(Metal_PSD[0][Metal_PSD[1]<10])
        Favourite_PSD_Average[:,i-1]+=10*np.log10(Favourite_PSD[0][Favourite_PSD[1]<10])
        plt.clf()
    writer.save()
    row=row+1
    plt.clf()
Metal_PSD_Average=Metal_PSD_Average/len(dir_list)
Favourite_PSD_Average=Favourite_PSD_Average/len(dir_list)
workbook.close() 
Metal_Average=pd.DataFrame(Metal_PSD_Average)
Metal_Average.columns=param_full[1:16]
Metal_Average.to_csv(r'PSD_IMU_Data/onemin/AverageMetalPSD.csv')
Favourite_Average=pd.DataFrame(Favourite_PSD_Average)
Favourite_Average.columns=param_full[1:16]
Favourite_Average.to_csv(r'PSD_IMU_Data/onemin/AverageFavouritePSD.csv')
for j in Metal_Average.columns:
    fig=plt.figure()
    plt.plot(Metal_PSD[1][Metal_PSD[1]<10],(Metal_Average[j]),'r')
    plt.plot(Metal_PSD[1][Metal_PSD[1]<10],(Favourite_Average[j]),'b')
    plt.legend(('Stressed','Relaxed'))
    plt.title(j)
    plt.xlabel('Frequency')
    plt.ylabel(j+'(in dB)')
    plt.savefig('PSD_IMU_Data/onemin/MeanPSD/mean_of_log/'+j+'.png')


['Resampled_IMU_Head/B300.xlsx', 'Resampled_IMU_Head/B301.xlsx', 'Resampled_IMU_Head/B303.xlsx', 'Resampled_IMU_Head/B304.xlsx', 'Resampled_IMU_Head/B305.xlsx', 'Resampled_IMU_Head/B306.xlsx', 'Resampled_IMU_Head/B307.xlsx', 'Resampled_IMU_Head/B308.xlsx', 'Resampled_IMU_Head/B309.xlsx', 'Resampled_IMU_Head/B311.xlsx', 'Resampled_IMU_Head/B312.xlsx', 'Resampled_IMU_Head/B313.xlsx', 'Resampled_IMU_Head/B314.xlsx', 'Resampled_IMU_Head/B315.xlsx', 'Resampled_IMU_Head/B316.xlsx', 'Resampled_IMU_Head/B317.xlsx', 'Resampled_IMU_Head/B318.xlsx', 'Resampled_IMU_Head/B319.xlsx', 'Resampled_IMU_Head/B320.xlsx', 'Resampled_IMU_Head/B321.xlsx', 'Resampled_IMU_Head/B322.xlsx', 'Resampled_IMU_Head/B324.xlsx', 'Resampled_IMU_Head/B325.xlsx', 'Resampled_IMU_Head/B326.xlsx']


/usr/local/lib/python3.5/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>